In [25]:
# Python + Machine Learning

# Cours 3: Travail avec les séquences et extraction d'informations de bases de données biologiques.

---

## 3 - 0 Rappels

In [72]:
def parse(arg):
    file = open(arg, "r")
    target = open("output.txt", "w")

    for line in file.readlines():
        for letter in line:
            if letter not in "aoieuy":
                target.write(letter + "\n")
    
    file.close()
    target.close()

print(parse("input.txt"))

None


In [47]:
f_output = open("output.txt", "r")

print(f_output.read())

f_output.close()

>
S
q
n
c
 
1



## 3 - 1 Regex et travail sur les séquences

### 3 - 1 - 1 Regex

Permet de trouver une structure/pattern dans une séquence de façon efficiente. Vous pouvez vous entraîner [ici](http://regexr.com/) sur les regex.

**Bases**

- Any of a, b, c

    ```[abc]```


- Not a, b, or c

    ```[^abc]```


- Any between a and g

    ```[a-g]```


- Any between a and g, and 1 and 3

    ```[a-g1-3]```
    
    
- Any except newline

    ```.```
    

- Soit GC, soit CG

    ```GC|CG```

In [180]:
from re import findall                          # module regex

seq = "GGAAAAAAAAAGCaaaCGaaaaaCCaaaaaGPaaaaGUaa\n\n\naaaW1w1W1W1"

pattern = "G[GC]"                               # G suivi d'un G ou C
print(pattern, "=>", findall(pattern, seq), "\n")

pattern = "G[^CP]"                              # G suivi d'un AA autre que C ou P
print(pattern, "=>", findall(pattern, seq), "\n")

pattern = "."                                   # Tout sauf un new line
print(pattern, "=>", findall(pattern, seq), "\n")

pattern = "[a-z0-9]"                            # Lettre minuscule ou chiffre
print(pattern, "=>", findall(pattern, seq), "\n")

pattern = "GC|CG"                               # Soit GC, soit CG
print(pattern, "=>", findall(pattern, seq), "\n")

G[GC] => ['GG', 'GC'] 

G[^CP] => ['GG', 'Ga', 'GU'] 

. => ['G', 'G', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'G', 'C', 'a', 'a', 'a', 'C', 'G', 'a', 'a', 'a', 'a', 'a', 'C', 'C', 'a', 'a', 'a', 'a', 'a', 'G', 'P', 'a', 'a', 'a', 'a', 'G', 'U', 'a', 'a', 'a', 'a', 'a', 'W', '1', 'w', '1', 'W', '1', 'W', '1'] 

[a-z0-9] => ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', '1', 'w', '1', '1', '1'] 

GC|CG => ['GC', 'CG'] 



**Quantifiers**

- \>=0
    
>   ```a*```


- \>=1
    
>   ```a+```


- 0 or 1
    
>   ```a?```


- Exactly 5
    
>   ```a{5}```


- \>=2
    
>   ```a{2,}```


- Between one & three
    
>   ```a{1,3}```


- Match as few as possible

>   ```a+? a{2,}?```

In [119]:
from re import findall

seq = "aaapauaaaaaaaar"
pattern = "a+"                                  # Tous les a qu'on peut prendre
print(pattern, "=>", findall(pattern, seq), "\n")


seq = "GGGaaaaaaaaaaaGPGaaaaaaaaaaaaaaGPPGCaaaaaaaaaaaaaaGPCCC"
pattern = "GP?[GC]+"                            # G suivi optionnellement d'un P suivi obligatoirement d'un ou plus G ou C
print(pattern, "=>", findall(pattern, seq), "\n")


seq = "AAAAAAAAAAAA\nAAAAAAAAA"
pattern = ".+"                                  # Tout sauf new line (greedy)
print(pattern, "=>", findall(pattern, seq), "\n")

seq = "GCGGCGCCCCGCGCGCC\nGCGCGCGGGGGGG"
pattern = "[GC][GC]"                            # Un G ou un C suivi d'un G ou un C
# pattern = "[GC]{2}"
print(pattern, "=>", findall(pattern, seq), "\n")

a+ => ['aaa', 'a', 'aaaaaaaa'] 

GP?[GC]+ => ['GGG', 'GPG', 'GC', 'GPCCC'] 

.+ => ['AAAAAAAAAAAA', 'AAAAAAAAA'] 

[GC][GC] => ['GC', 'GG', 'CG', 'CC', 'CC', 'GC', 'GC', 'GC', 'GC', 'GC', 'GC', 'GG', 'GG', 'GG'] 



Equivalent d'écrire:

```[0-9]``` et ```\d``` (**any digit**)

```[A-Za-z0-9]``` et ```\w``` (**any word**)

In [120]:
from re import findall

seq = "3 feuilles A22 blanc-noir"

# print(findall("\w", seq))
# print(findall("\d", seq))
print(findall("\w+", seq))
print(findall("\d+", seq))

['3', 'feuilles', 'A22', 'blanc', 'noir']
['3', '22']


**findall** donne la liste des sous-seqs d'une seq qui correspondent à un pattern.

La fonction **finditer** nous permet d'avoir des informations sur ces matchs.

In [149]:
from re import finditer

seq = "GGaaaaaaaaGCaaaaaaGPaaaaGU"
pattern = "G[GC]"                               # G suivi d'un G ou C

for i in finditer(pattern, seq):
    print(i)                                    # Cette fonction retourne un objet de type match
    
for i in finditer(pattern, seq):
    print(i.group(), i.start())                 # E.g. on peut avoir la sous-seq matchée + start

<_sre.SRE_Match object; span=(0, 2), match='GG'>
<_sre.SRE_Match object; span=(10, 12), match='GC'>
GG 0
GC 10


On peut définir des **groupes de capture**.

Dans les patterns que nous avions avant, toute la pattern correspondait à un groupe de capture.

Ici, on a besoin de définir un groupe pour chercher les P suivis de soit GC soit CG.

In [203]:
from re import findall, finditer

seq = "PCGaaaaaaaaaaPGCaaaaaaaaaPGGaaaaaaaaaaPGCGaaaaaaaPGCGCCG"
pattern = "P(GC|CG)+"

print(findall(pattern, seq))

for i in finditer(pattern, seq):
    print(i.span(), i.group(0), i.group(1))

['CG', 'GC', 'GC', 'CG']
(0, 3) PCG CG
(13, 16) PGC GC
(38, 41) PGC GC
(49, 56) PGCGCCG CG


On remarque que les matchs findall =/= les matchs finditer: c'est une question de groupe explicite (groupe 0 = tout, groupe 1 = groupe explicite), mais c'est trop complexe, donc utilisez juste finditer pour plus d'options.

### 3 - 1 - 2 Exercices

Chercher les paramètres statistiques dans un texte.

In [229]:
from re import findall

f = open("statistique.txt", "r")
seq = f.read()
print(seq, "\n")

# pattern = "\[[^\[\]]+\]^\]"           # La pattern vue en cours
pattern = "\[[^\[\]]+\][^\]]"           # La pattern qui fonctionne =>
                                        # crochet qui ouvre, >=1 non-crochets, crochet qui ferme, non-crochet
                                        # Le dernier "non crochet" n'était pas entre crochets
                                        # Les "non-machin" doivent être entre crochets,
                                        # comme on l'avait fait pour les autres

print(findall(pattern, seq))            # Remarquez qu'on prend le machin qui suit le dernier crochet

Un machin qui doit être pris [R2 = .1, F(1, 2) = 3] suivi d'un machin [[]] qui ne doit pas être pris. Un autre bon machin [t2 = 1, N(0, 1) = 2, Z = 3]. Une phrase avec un machin [R = .2, F(1, 2) = 5]] presque bon. [var = 3, sd = 1.7] encore un bon machin. 

['[R2 = .1, F(1, 2) = 3] ', '[t2 = 1, N(0, 1) = 2, Z = 3].', '[var = 3, sd = 1.7] ']


Obtenir seq continue à partir de seq format genbank.

In [187]:
from re import findall

f = open("seq_gb.txt", "r")
pattern = "[A-Za-z]+"                   # On cherche toutes les lettres

seqs = findall(pattern, f.read())
print(seqs, type(seqs))

seq = "".join(seqs)

print(seq)

['GPUCPUGPUC', 'GPUCGPUCGP', 'GPU'] <class 'list'>
GPUCPUGPUCGPUCGPUCGPGPU


Chercher tous les codons stop dans une seq.

In [207]:
from re import finditer

f = open("seq_fasta.txt", "r")
seq = f.read()

pattern = "U(AA|AG|GA)"

stops = []
for i in finditer(pattern, seq):
    stops.append(i.start())
    stops.append(i.group())

print(stops)

[34, 'UAG', 45, 'UAG', 58, 'UAG', 61, 'UAG', 71, 'UGA', 94, 'UAA', 115, 'UAA', 124, 'UAG', 154, 'UGA', 215, 'UAA', 219, 'UAA', 222, 'UGA', 230, 'UGA', 280, 'UGA', 286, 'UAG', 292, 'UGA', 316, 'UAA', 331, 'UGA', 381, 'UAA', 402, 'UAA', 405, 'UAA', 438, 'UAA', 457, 'UAA', 522, 'UGA', 564, 'UAA', 567, 'UAA', 589, 'UGA', 613, 'UAG', 649, 'UAA', 663, 'UAG', 695, 'UGA', 700, 'UGA', 717, 'UAA', 746, 'UAA', 755, 'UAA', 758, 'UAA', 762, 'UAA', 818, 'UGA', 859, 'UGA', 897, 'UGA', 902, 'UGA', 951, 'UGA', 961, 'UAA', 1002, 'UAG', 1006, 'UGA', 1009, 'UAA']


## 3 - 2 Web scraping

### 3 - 2 - 1 Extraction automatisée de contenu des bases de données biologiques.

Quand vous voulez écrire un programme pour extraire des données sur internet, il faut repérer l'endroit où elles se trouvent via le code source de la page.

Ensuite déterminez le xpath soit en téléchargeant une extension qui le fait pour vous sur votre navigateur, soit en suivant les instructions [ici](http://www.freeformatter.com/xpath-tester.html#xpath-examples). Il faut repérer la balise dans laquelle se trouve votre info (type, classe, id, etc.). Inspect element =/= code source, il faut toujours regarder le code source.

Les sites de NCBI sont accessibles seulement via des modules spécialisés, comme BioPython (le code source contient du code javascript qui affiche les informations dans un navigateur tout en bloquant le scraping via html).

In [246]:
from requests import get
from lxml import html
from re import findall

query = get("http://www.uniprot.org/uniprot/P15442")
# On récupère la page sous format non exploitable

print(type(query), type(query.content), "\n", query.content[:1000], "\n")
# Les 1000 premiers caractères du contenu de query

query_html = html.fromstring(query.content)
print(type(query_html), "\n")
# On convertit le format en html

seqs_raw = query_html.xpath("(//pre[@class = 'sequence'])[1]/text()")
# On utilise le xpath:
# //pre => toutes les balises pre
# [@class = "sequence"] => qui ont une propriété class = "sequence"
# [1] => on prend la première balise qui vérifie les conditions précédentes
# /text() => on prend le texte contenu dans cet élément
print(seqs_raw[:10], "\n")

seq_raw = "".join(seqs_raw)
pattern = "[A-Z]+"

seq = findall(pattern, seq_raw)
print("".join(seq))

<class 'requests.models.Response'> <class 'bytes'> 
 b'<!DOCTYPE html SYSTEM "about:legacy-compat">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en"><head><title>GCN2 - eIF-2-alpha kinase GCN2 - Saccharomyces cerevisiae (strain ATCC 204508 / S288c) (Baker&#039;s yeast) - GCN2 gene &amp; protein</title><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link href="/" rel="home"/><link href="http://creativecommons.org/licenses/by-nd/3.0/" rel="license"/><link type="image/vnd.microsoft.icon" href="/favicon.ico" rel="shortcut icon"/><link href="http://ebi-uniprot.github.io/CDN/protvista/css/protvista.css" rel="stylesheet"/><link href="/css-compr.css2016_09" type="text/css" rel="stylesheet"/><script type="text/javascript">\n\t\t\tvar BASE = \'/\';\n\t\t\tvar image_x_inverse = \'/images/x_inverse.png\';\n\t\t</script>\n\t\t<!--[if lte 

### 3 - 2 - 2 Exercices

Extraction de données.

In [278]:
from requests import get
from lxml import html

query = get("http://planetpython.org/")         # De "from requests" à "xpath", ce sera toujours le même code
query_html = html.fromstring(query.content)

titles = query_html.xpath("(//h4/a)/text()")    # Le texte de tous les titres sur planetpython.com
                                                # Ca ressemble à ça: <h4><a ...>Texte</a></h4>
print(len(titles), "\n", titles[-4])

25 
 NumPy Tutorial: Data analysis with Python
